## Bevezetés 

Ez a lecke a következőket fogja lefedni: 
- Mi az a függvényhívás és milyen esetekben használható 
- Hogyan lehet függvényhívást létrehozni az OpenAI segítségével 
- Hogyan lehet egy függvényhívást integrálni egy alkalmazásba 

## Tanulási célok 

A lecke elvégzése után tudni fogod és megérted: 

- A függvényhívás használatának célját 
- A függvényhívás beállítását az OpenAI szolgáltatás segítségével 
- Hatékony függvényhívások tervezését az alkalmazásod használati esetéhez


## Függvényhívások megértése

Ehhez a leckéhez egy olyan funkciót szeretnénk létrehozni az oktatási startupunk számára, amely lehetővé teszi a felhasználók számára, hogy egy chatbot segítségével technikai tanfolyamokat találjanak. Olyan tanfolyamokat fogunk ajánlani, amelyek megfelelnek a képességszintjüknek, jelenlegi szerepüknek és az érdeklődési technológiának.

Ehhez a következő kombinációt fogjuk használni:
 - `OpenAI` a felhasználó számára egy csevegési élmény létrehozásához
 - `Microsoft Learn Catalog API` a felhasználók segítésére a tanfolyamok megtalálásában a felhasználó kérésének alapján
 - `Function Calling` a felhasználó lekérdezésének átvételéhez és egy függvényhez való elküldéséhez az API kérés végrehajtásához.

A kezdéshez nézzük meg, miért szeretnénk egyáltalán függvényhívást használni:

print("Üzenetek a következő kérésben:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # új választ kapunk a GPT-től, ahol láthatja a függvény válaszát


print(second_response.choices[0].message)


### Miért a függvényhívás

Ha elvégeztél bármely más leckét ebben a tanfolyamban, valószínűleg érted a Nagy Nyelvi Modellek (LLM-ek) használatának erejét. Remélhetőleg a korlátaikat is látod.

A függvényhívás az OpenAI Szolgáltatás egy olyan funkciója, amely a következő kihívások kezelésére készült:

Válaszformátumok következetlensége:
- A függvényhívás előtt a nagy nyelvi modell válaszai strukturálatlanok és következetlenek voltak. A fejlesztőknek bonyolult érvényesítő kódot kellett írniuk az egyes kimeneti variációk kezelésére.

Korlátozott integráció külső adatokkal:
- E funkció előtt nehéz volt az alkalmazás más részeiből származó adatokat beépíteni egy csevegési kontextusba.

A válaszformátumok szabványosításával és a külső adatok zökkenőmentes integrációjának lehetővé tételével a függvényhívás egyszerűsíti a fejlesztést, és csökkenti a további érvényesítési logika szükségességét.

A felhasználók nem kaphattak olyan válaszokat, mint például „Milyen az aktuális időjárás Stockholmban?”. Ennek oka, hogy a modellek csak a betanításuk idejéig rendelkeztek adatokkal.

Nézzük meg az alábbi példát, amely illusztrálja ezt a problémát:

Tegyük fel, hogy létre akarunk hozni egy hallgatói adatbázist, hogy a megfelelő kurzust javasolhassuk nekik. Lent két olyan hallgató leírását látjuk, amelyek nagyon hasonló adatokat tartalmaznak.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Ezt szeretnénk elküldeni egy LLM-nek az adatok elemzéséhez. Ezt később felhasználhatjuk az alkalmazásunkban, hogy elküldjük egy API-nak vagy eltároljuk egy adatbázisban.

Hozzunk létre két azonos promptot, amelyekben utasítjuk az LLM-et, hogy milyen információk érdekelnek minket:


Ezt egy LLM-nek szeretnénk elküldeni, hogy elemezze a termékünk szempontjából fontos részeket. Így két azonos promptot hozhatunk létre az LLM utasításához:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Miután elkészítettük ezt a két promptot, elküldjük őket az LLM-nek az `openai.ChatCompletion` használatával. A promptot a `messages` változóban tároljuk, és a szerepet `user`-re állítjuk. Ez azért van, hogy utánozzuk egy felhasználó üzenetének írását egy chatbot számára.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Most már mindkét kérést elküldhetjük az LLM-nek, és megvizsgálhatjuk a kapott választ.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Bár a promptok ugyanazok és a leírások hasonlóak, a `Grades` tulajdonság különböző formátumokban jelenhet meg.

Ha többször lefuttatod a fenti cellát, a formátum lehet `3.7` vagy `3.7 GPA`.

Ennek az az oka, hogy az LLM nem strukturált adatokat vesz be a megírt prompt formájában, és szintén nem strukturált adatokat ad vissza. Szükségünk van egy strukturált formátumra, hogy tudjuk, mire számíthatunk az adatok tárolásakor vagy használatakor.

Funkcionális hívás használatával biztosíthatjuk, hogy strukturált adatokat kapjunk vissza. Funkcionális hívás esetén az LLM valójában nem hív meg vagy futtat le semmilyen függvényt. Ehelyett létrehozunk egy struktúrát, amelyet az LLM követ a válaszai során. Ezután ezeket a strukturált válaszokat használjuk annak meghatározására, hogy melyik függvényt futtassuk az alkalmazásainkban.


![Függvényhívás folyamata](../../../../translated_images/Function-Flow.083875364af4f4bb.hu.png)


Ezután elvehetjük, amit a függvény visszaad, és visszaküldhetjük ezt az LLM-nek. Az LLM ezután természetes nyelven válaszol a felhasználó kérdésére.


### Funkcióhívások használati esetei

**Külső eszközök hívása**  
A chatbotok nagyszerűek arra, hogy válaszokat adjanak a felhasználók kérdéseire. A funkcióhívás használatával a chatbotok a felhasználók üzeneteit felhasználva bizonyos feladatokat végezhetnek el. Például egy diák megkérheti a chatbotot, hogy „Küldj e-mailt az oktatómnak, hogy több segítségre van szükségem ebben a témában”. Ez egy `send_email(to: string, body: string)` nevű funkcióhívást eredményezhet.

**API vagy adatbázis lekérdezések létrehozása**  
A felhasználók természetes nyelv használatával találhatnak információkat, amelyeket formázott lekérdezéssé vagy API-kéréssé alakítanak át. Ennek példája lehet egy tanár, aki megkérdezi: „Kik azok a diákok, akik teljesítették az utolsó feladatot”, ami egy `get_completed(student_name: string, assignment: int, current_status: string)` nevű funkcióhívást indíthat.

**Strukturált adatok létrehozása**  
A felhasználók egy szövegrészt vagy CSV-t vehetnek, és az LLM segítségével fontos információkat nyerhetnek ki belőle. Például egy diák egy Wikipédia cikket alakíthat át a békemegállapodásokról, hogy AI villámkártyákat készítsen. Ezt egy `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` nevű funkció használatával lehet megtenni.


## 2. Az első függvényhívás létrehozása

A függvényhívás létrehozásának folyamata 3 fő lépésből áll:  
1. A Chat Completions API hívása a függvényeid listájával és egy felhasználói üzenettel  
2. A modell válaszának elolvasása egy művelet végrehajtásához, pl. egy függvény vagy API hívás végrehajtása  
3. Egy újabb hívás végrehajtása a Chat Completions API-hoz a függvényed válaszával, hogy ezt az információt felhasználva válaszolj a felhasználónak.


![Egy függvényhívás folyamata](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.hu.png)


### Egy függvényhívás elemei

#### Felhasználói bemenet

Az első lépés egy felhasználói üzenet létrehozása. Ezt dinamikusan is hozzárendelhetjük egy szövegbeviteli mező értékének felhasználásával, vagy itt is megadhatunk egy értéket. Ha ez az első alkalom, hogy a Chat Completions API-val dolgozol, meg kell határoznunk az üzenet `role` és `content` értékét.

A `role` lehet `system` (szabályok létrehozása), `assistant` (a modell) vagy `user` (a végfelhasználó). A függvényhíváshoz ezt `user`-nek állítjuk be, és egy példa kérdést adunk meg.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Függvények létrehozása.

Ezután definiálunk egy függvényt és annak paramétereit. Itt csak egy függvényt fogunk használni, amelynek neve `search_courses`, de több függvényt is létrehozhatsz.

**Fontos** : A függvények be vannak építve a rendszerüzenetbe az LLM számára, és beleszámítanak a rendelkezésre álló tokenek mennyiségébe.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definíciók** 

A függvénydefiníció szerkezete több szintből áll, mindegyiknek megvannak a saját tulajdonságai. Íme a beágyazott szerkezet bontása:

**Felső szintű függvénytulajdonságok:**

`name` - A függvény neve, amelyet hívni szeretnénk. 

`description` - Ez a függvény működésének leírása. Itt fontos, hogy pontos és világos legyen. 

`parameters` - Egy lista azokról az értékekről és formátumról, amelyeket a modellnek a válaszában elő kell állítania. 

**Paraméter objektum tulajdonságai:**

`type` - A paraméter objektum adattípusa (általában "object")

`properties` - A konkrét értékek listája, amelyeket a modell a válaszához használni fog. 

**Egyedi paraméter tulajdonságok:**

`name` - Implicit módon a tulajdonság kulcsa határozza meg (pl. "role", "product", "level")

`type` - Ennek a konkrét paraméternek az adattípusa (pl. "string", "number", "boolean") 

`description` - A konkrét paraméter leírása 

**Opcionális tulajdonságok:**

`required` - Egy tömb, amely felsorolja, hogy mely paraméterek szükségesek a függvényhívás teljesítéséhez. 


### A függvényhívás elkészítése  
Miután definiáltuk a függvényt, most be kell illesztenünk azt a Chat Completion API hívásába. Ezt úgy tesszük meg, hogy hozzáadjuk a `functions`-t a kéréshez. Ebben az esetben `functions=functions`.  

Van egy lehetőség arra is, hogy a `function_call` értékét `auto`-ra állítsuk. Ez azt jelenti, hogy az LLM döntheti el, melyik függvényt kell meghívni a felhasználói üzenet alapján, ahelyett, hogy mi rendelnénk hozzá.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Most nézzük meg a választ, és lássuk, hogyan van formázva:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Látható, hogy a függvény neve meg van hívva, és a felhasználói üzenet alapján a LLM képes volt megtalálni az adatokat, hogy illeszkedjenek a függvény argumentumaihoz.


## 3. Függvényhívások integrálása egy alkalmazásba. 


Miután teszteltük a formázott választ az LLM-től, most integrálhatjuk ezt egy alkalmazásba. 

### A folyamat kezelése 

Ahhoz, hogy ezt integráljuk az alkalmazásunkba, tegyük meg a következő lépéseket: 

Először hívjuk meg az OpenAI szolgáltatásokat, és tároljuk az üzenetet egy `response_message` nevű változóban. 


In [ ]:
response_message = response.choices[0].message

Most definiáljuk azt a függvényt, amely meghívja a Microsoft Learn API-t, hogy lekérje a tanfolyamok listáját:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Legjobb gyakorlatként ezután megnézzük, hogy a modell szeretne-e egy függvényt hívni. Ezután létrehozunk egy elérhető függvényt, és összepárosítjuk azt a hívott függvénnyel.  
Ezután a függvény argumentumait leképezzük az LLM argumentumaira.

Végül hozzáfűzzük a függvényhívás üzenetét és azokat az értékeket, amelyeket a `search_courses` üzenet adott vissza. Ez megadja az LLM-nek az összes szükséges információt ahhoz, hogy természetes nyelven válaszoljon a felhasználónak.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Most elküldjük a frissített üzenetet az LLM-nek, hogy természetes nyelvű választ kapjunk az API JSON formátumú válasza helyett.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kód kihívás

Nagyszerű munka! Az OpenAI Function Calling tanulásának folytatásához elkészítheted: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - A függvény további paraméterei, amelyek segíthetnek a tanulóknak több tanfolyam megtalálásában. A rendelkezésre álló API paramétereket itt találod:  
 - Készíts egy másik függvényhívást, amely több információt kér a tanulótól, például az anyanyelvét  
 - Készíts hibakezelést arra az esetre, ha a függvényhívás és/vagy az API hívás nem ad vissza megfelelő tanfolyamokat  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Jogi nyilatkozat**:
Ezt a dokumentumot az AI fordító szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével fordítottuk le. Bár a pontosságra törekszünk, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az anyanyelvén tekintendő hiteles forrásnak. Fontos információk esetén szakmai, emberi fordítást javaslunk. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
